##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## 关于如何使用模型压缩工具
代码关于模型权重剪枝，tensorflow模型的优化工具

权重剪枝的作用：1、模型压缩；2、运行加速。

关于keras的权重剪枝api，可以在训练过程中对模型进行剪枝，或对预训练模型进行剪枝。

这个api同样可以在TensorFlow Lite场景下使用。

## 安装环境 

使用剪枝  `tensorflow-model-optimization` 包的API.详细见 [TensorFlow model optimization repo](https://github.com/tensorflow/model-optimization) 

In [1]:
# ! pip uninstall -y tensorflow
# ! pip uninstall -y tf-nightly
# ! pip install -U tensorflow-gpu==1.14.0

# ! pip install tensorflow-model-optimization

In [1]:
%load_ext tensorboard
import tensorboard

In [2]:
import tensorflow as tf
tf.enable_eager_execution()

import tempfile
import zipfile
import os

D:\ProgramData\Anaconda2\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 加载mnist数据

In [3]:
batch_size = 128
num_classes = 10
epochs = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

if tf.keras.backend.image_data_format() == 'channels_first':
  x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
  input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## 创建模型

In [4]:
l = tf.keras.layers

model = tf.keras.Sequential([
    l.Conv2D(
        32, 5, padding='same', activation='relu', input_shape=input_shape),
    l.MaxPooling2D((2, 2), (2, 2), padding='same'),
    l.BatchNormalization(),
    l.Conv2D(64, 5, padding='same', activation='relu'),
    l.MaxPooling2D((2, 2), (2, 2), padding='same'),
    l.Flatten(),
    l.Dense(1024, activation='relu'),
    l.Dropout(0.4),
    l.Dense(num_classes, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              3

### 训练模型， accuracy >99%

In [7]:
logdir="./tensorboard_logs/logs"
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch=0)]

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 0.2900 - acc: 0.9442 - val_loss: 0.0854 - val_acc: 0.9850
Epoch 2/10
60000/60000 [==============================] - 75s 1ms/sample - loss: 0.0458 - acc: 0.9856 - val_loss: 0.0276 - val_acc: 0.9909
Epoch 3/10
60000/60000 [==============================] - 75s 1ms/sample - loss: 0.0324 - acc: 0.9898 - val_loss: 0.0410 - val_acc: 0.9867
Epoch 4/10
60000/60000 [==============================] - 75s 1ms/sample - loss: 0.0239 - acc: 0.9923 - val_loss: 0.0343 - val_acc: 0.9897
Epoch 5/10
60000/60000 [==============================] - 76s 1ms/sample - loss: 0.0220 - acc: 0.9926 - val_loss: 0.0254 - val_acc: 0.9930
Epoch 6/10
60000/60000 [==============================] - 74s 1ms/sample - loss: 0.0188 - acc: 0.9940 - val_loss: 0.0269 - val_acc: 0.9916
Epoch 7/10
60000/60000

### 保存原始模型，以方便后续对比

In [8]:
# 在临时目录下保存模型文件
_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Saving model to:  C:\Users\ADMINI~1\AppData\Local\Temp\tmpvnea6y58.h5


## 训练一个剪枝模型

可以使用 `prune_low_magnitude()` API 对模型进行剪枝. 这个api可以应用与单个网络层或者是整个网络模型。

可以设立参数配置：将以75%的稀疏性为目标，从步骤2000开始，每100步（也称为epoch）修剪一次权重。

### 建立基于单个网络层剪枝的模型
In this example, we show how to use the API at the level of layers, and build a pruned MNIST solver model.

In this case, the `prune_low_magnitude(`) 
receives as parameter the Keras layer whose weights we want pruned.

This function requires a pruning params which configures the pruning algorithm during training. Please refer to our github page for detailed documentation. The parameter used here means:


1.   **Sparsity.** PolynomialDecay is used across the whole training process. We start at the sparsity level 50% and gradually train the model to reach 90% sparsity. X% sparsity means that X% of the weight tensor is going to be pruned away.
2.   **Schedule**. Connections are pruned starting from step 2000 to the end of training, and runs every 100 steps. The reasoning behind this is that we want to train the model without pruning for a few epochs to reach a certain accuracy, to aid convergence. Furthermore, we give the model some time to recover after each pruning step, so pruning does not happen on every step. We set the pruning frequency to 100.



In [9]:
from tensorflow_model_optimization.sparsity import keras as sparsity

四个重要的参数： begin_sparsity, final_sparsity, begin_step and end_step。其中，end_step参数需要根据训练数据的数量、batch size和pochs来确定。

下面定义这四个重要参数：

In [10]:
import numpy as np

epochs = 12
num_train_samples = x_train.shape[0]
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print('End step: ' + str(end_step))

End step: 5628


In [11]:
pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=2000,
                                                   end_step=end_step,
                                                   frequency=100)
}

pruned_model = tf.keras.Sequential([
    sparsity.prune_low_magnitude(
        l.Conv2D(32, 5, padding='same', activation='relu'),
        input_shape=input_shape,
        **pruning_params),
    l.MaxPooling2D((2, 2), (2, 2), padding='same'),
    l.BatchNormalization(),
    sparsity.prune_low_magnitude(
        l.Conv2D(64, 5, padding='same', activation='relu'), **pruning_params),
    l.MaxPooling2D((2, 2), (2, 2), padding='same'),
    l.Flatten(),
    sparsity.prune_low_magnitude(l.Dense(1024, activation='relu'),
                                 **pruning_params),
    l.Dropout(0.4),
    sparsity.prune_low_magnitude(l.Dense(num_classes, activation='softmax'),
                                 **pruning_params)
])

pruned_model.summary()

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d_2 (None, 28, 28, 32)        1634      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
prune_low_magnitude_conv2d_3 (None, 14, 14, 64)        102466    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
__________________________________________

### 训练剪枝模型

Start pruning from step 2000 when accuracy >98%

In [15]:
logdir="./tensorboard_logs/pruning_training_logs"
pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

pruned_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))

score = pruned_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
59904/60000 [============================>.] - ETA: 0s - loss: 0.2545 - acc: 0.9423INFO:tensorflow:Summary name prune_low_magnitude_conv2d_2/mask:0/sparsity is illegal; using prune_low_magnitude_conv2d_2/mask_0/sparsity instead.
INFO:tensorflow:Summary name prune_low_magnitude_dense_2/threshold:0/threshold is illegal; using prune_low_magnitude_dense_2/threshold_0/threshold instead.
INFO:tensorflow:Summary name prune_low_magnitude_conv2d_2/threshold:0/threshold is illegal; using prune_low_magnitude_conv2d_2/threshold_0/threshold instead.
INFO:tensorflow:Summary name prune_low_magnitude_dense_3/mask:0/sparsity is illegal; using prune_low_magnitude_dense_3/mask_0/sparsity instead.
INFO:tensorflow:Summary name prune_low_magnitude_dense_3/threshold:0/threshold is illegal; using prune_low_magnitude_dense_3/threshold_0/threshold instead.
INFO:tensorflow:Summary name prune_low_magnitude_conv2d_3/mask:0/sparsity is illegal; using prun

INFO:tensorflow:Summary name prune_low_magnitude_dense_2/mask:0/sparsity is illegal; using prune_low_magnitude_dense_2/mask_0/sparsity instead.
INFO:tensorflow:Summary name prune_low_magnitude_conv2d_3/threshold:0/threshold is illegal; using prune_low_magnitude_conv2d_3/threshold_0/threshold instead.
60000/60000 [==============================] - 87s 1ms/sample - loss: 0.0159 - acc: 0.9949 - val_loss: 0.0239 - val_acc: 0.9928
Epoch 7/10
59904/60000 [============================>.] - ETA: 0s - loss: 0.0132 - acc: 0.9957INFO:tensorflow:Summary name prune_low_magnitude_conv2d_2/mask:0/sparsity is illegal; using prune_low_magnitude_conv2d_2/mask_0/sparsity instead.
INFO:tensorflow:Summary name prune_low_magnitude_dense_2/threshold:0/threshold is illegal; using prune_low_magnitude_dense_2/threshold_0/threshold instead.
INFO:tensorflow:Summary name prune_low_magnitude_conv2d_2/threshold:0/threshold is illegal; using prune_low_magnitude_conv2d_2/threshold_0/threshold instead.
INFO:tensorflow:

### 保存并加载剪枝模型

加载完剪枝模型后，再训练两个epochs:

In [16]:
_, checkpoint_file = tempfile.mkstemp('.h5')
print('Saving pruned model to: ', checkpoint_file)
# saved_model() sets include_optimizer to True by default. Spelling it out here
# to highlight.
tf.keras.models.save_model(pruned_model, checkpoint_file, include_optimizer=True)

with sparsity.prune_scope():
  restored_model = tf.keras.models.load_model(checkpoint_file)

restored_model.fit(x_train, y_train,
                   batch_size=batch_size,
                   epochs=2,
                   verbose=1,
                   callbacks=callbacks,
                   validation_data=(x_test, y_test))

score = restored_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Saving pruned model to:  C:\Users\ADMINI~1\AppData\Local\Temp\tmpr0fljmte.h5
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
59904/60000 [============================>.] - ETA: 0s - loss: 0.0109 - acc: 0.9966INFO:tensorflow:Summary name prune_low_magnitude_dense_2_1/threshold:0/threshold is illegal; using prune_low_magnitude_dense_2_1/threshold_0/threshold instead.
INFO:tensorflow:Summary name prune_low_magnitude_dense_2_1/mask:0/sparsity is illegal; using prune_low_magnitude_dense_2_1/mask_0/sparsity instead.
INFO:tensorflow:Summary name prune_low_magnitude_dense_3_1/mask:0/sparsity is illegal; using prune_low_magnitude_dense_3_1/mask_0/sparsity instead.
INFO:tensorflow:Summary name prune_low_magnitude_dense_3_1/threshold:0/threshold is illegal; using prune_low_magnitude_dense_3_1/threshold_0/threshold instead.
INFO:tensorflow:Summary name prune_low_magnitude_conv2d_2_1/mask:0/sparsity is illegal; using prune_low_magnitude_conv2d_2_1/mask_0/sparsity instead.
INFO:tensorflo

需要注意的是

*  保存模型要将参数 include_optimizer 设置为 True. 
*   加载模型的时候，需要使用 prune_scope() 进行反序列化



### 使用剪枝模型之前，需要从剪枝模型中去除剪枝模块

使用strip_pruning` API 

In [17]:
final_model = sparsity.strip_pruning(pruned_model)
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)             

In [18]:
_, pruned_keras_file = tempfile.mkstemp('.h5')
print('Saving pruned model to: ', pruned_keras_file)

# 不需要保存优化器模块
tf.keras.models.save_model(final_model, pruned_keras_file, include_optimizer=False)

Saving pruned model to:  C:\Users\ADMINI~1\AppData\Local\Temp\tmpocutxj12.h5


### Compare the size of the unpruned vs. pruned model after compression

In [19]:
_, zip1 = tempfile.mkstemp('.zip') 
with zipfile.ZipFile(zip1, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(keras_file)
print("Size of the unpruned model before compression: %.2f Mb" % 
      (os.path.getsize(keras_file) / float(2**20)))
print("Size of the unpruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip1) / float(2**20)))

_, zip2 = tempfile.mkstemp('.zip') 
with zipfile.ZipFile(zip2, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(pruned_keras_file)
print("Size of the pruned model before compression: %.2f Mb" % 
      (os.path.getsize(pruned_keras_file) / float(2**20)))
print("Size of the pruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip2) / float(2**20)))



Size of the unpruned model before compression: 12.52 Mb
Size of the unpruned model after compression: 11.59 Mb
Size of the pruned model before compression: 12.52 Mb
Size of the pruned model after compression: 2.51 Mb


### 对预训练模型进行剪枝

`prune_low_magnitude` 函数同样可以对整个keras模型进行剪枝。这时候，将是对所有网络层进行剪枝。

注意：
1、对于无法剪枝的官方网络层不起作用剪枝参数将被忽略,但对于未知的层（非官方网络层）的使用会报错。
2、对于不确定剪枝方案的网络层可以指定"un-pruned"参数。

对于预训练模型的剪枝需要进行 re-compile 。

Before we move forward with the example, lets address the common use case where you may already have a serialized pre-trained Keras model, which you would like to apply weight pruning on. We will take the original MNIST model trained previously to show how this works. In this case, you start by loading the model into memory like this:

In [20]:
# Load the serialized model
loaded_model = tf.keras.models.load_model(keras_file)

对预训练模型进行剪枝，需要从step 0开始.由于预训练模型已经达到我们想要的准确度，所以这里我们可以直接对模型剪枝，设置 begin_step = 0 训练4个 epochs.

In [21]:
epochs = 4
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print(end_step)

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(model, **new_pruning_params)
new_pruned_model.summary()

new_pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

1876
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d ( (None, 28, 28, 32)        1634      
_________________________________________________________________
prune_low_magnitude_max_pool (None, 14, 14, 32)        1         
_________________________________________________________________
prune_low_magnitude_batch_no (None, 14, 14, 32)        129       
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 14, 14, 64)        102466    
_________________________________________________________________
prune_low_magnitude_max_pool (None, 7, 7, 64)          1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 3136)              1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 1024)          

In [ ]:
logdir="./tensorboard_logs/pruning_pretrained_logs"


In [24]:
# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

new_pruned_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))

score = new_pruned_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/4
59904/60000 [============================>.] - ETA: 0s - loss: 0.0129 - acc: 0.9959INFO:tensorflow:Summary name prune_low_magnitude_dense/threshold:0/threshold is illegal; using prune_low_magnitude_dense/threshold_0/threshold instead.
INFO:tensorflow:Summary name prune_low_magnitude_conv2d_1/mask:0/sparsity is illegal; using prune_low_magnitude_conv2d_1/mask_0/sparsity instead.
INFO:tensorflow:Summary name prune_low_magnitude_conv2d_1/threshold:0/threshold is illegal; using prune_low_magnitude_conv2d_1/threshold_0/threshold instead.
INFO:tensorflow:Summary name prune_low_magnitude_dense_1/threshold:0/threshold is illegal; using prune_low_magnitude_dense_1/threshold_0/threshold instead.
INFO:tensorflow:Summary name prune_low_magnitude_conv2d/mask:0/sparsity is illegal; using prune_low_magnitude_conv2d/mask_0/sparsity instead.
INFO:tensorflow:Summary name prune_low_magnitude_conv2d/threshold:0/threshold is illegal; using prune_l

### Export the pruned model for serving

In [25]:
final_model = sparsity.strip_pruning(pruned_model)
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)             

In [26]:
_, new_pruned_keras_file = tempfile.mkstemp('.h5')
print('Saving pruned model to: ', new_pruned_keras_file)
tf.keras.models.save_model(final_model, new_pruned_keras_file, 
                        include_optimizer=False)

Saving pruned model to:  C:\Users\ADMINI~1\AppData\Local\Temp\tmpspcgd27o.h5


The model size after compression is the same as the one pruned layer-by-layer

In [27]:
_, zip3 = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip3, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(new_pruned_keras_file)
print("Size of the pruned model before compression: %.2f Mb" 
      % (os.path.getsize(new_pruned_keras_file) / float(2**20)))
print("Size of the pruned model after compression: %.2f Mb" 
      % (os.path.getsize(zip3) / float(2**20)))

Size of the pruned model before compression: 12.52 Mb
Size of the pruned model after compression: 2.51 Mb


## 转化成 TensorFlow Lite

转化需要使用 TFLiteConverter 的api:

### 使用TFLiteConverter转化模型

In [28]:
tflite_model_file = './sparse_mnist.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)
tflite_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Converted 12 variables to const ops.


### 对比 TensorFlow Lite 模型和压缩文件的大小

In [29]:
_, zip_tflite = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

Size of the tflite model before compression: 12.49 Mb
Size of the tflite model after compression: 2.42 Mb


### 评估TensorFlow Lite 模型的准确率

In [30]:
import numpy as np

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

def eval_model(interpreter, x_test, y_test):
  total_seen = 0
  num_correct = 0

  for img, label in zip(x_test, y_test):
    inp = img.reshape((1, 28, 28, 1))
    total_seen += 1
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_index)
    if np.argmax(predictions) == np.argmax(label):
      num_correct += 1

    if total_seen % 1000 == 0:
        print("Accuracy after %i images: %f" %
              (total_seen, float(num_correct) / float(total_seen)))

  return float(num_correct) / float(total_seen)

print(eval_model(interpreter, x_test, y_test))

Accuracy after 1000 images: 0.993000
Accuracy after 2000 images: 0.991000
Accuracy after 3000 images: 0.987333
Accuracy after 4000 images: 0.989000
Accuracy after 5000 images: 0.988800
Accuracy after 6000 images: 0.990333
Accuracy after 7000 images: 0.990714
Accuracy after 8000 images: 0.991875
Accuracy after 9000 images: 0.992556
Accuracy after 10000 images: 0.992200
0.9922


### 对 TensorFlow Lite模型的权重进行量化

可以对模型的参数进行8位精确度的量化转换可将大小缩减至原来的1/4.但是准确度有所降低。
原因：float为32位？


In [32]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

tflite_quant_model = converter.convert()

tflite_quant_model_file = './sparse_mnist_quant.tflite'
with open(tflite_quant_model_file, 'wb') as f:
  f.write(tflite_quant_model)

INFO:tensorflow:Converted 12 variables to const ops.


In [33]:
_, zip_tflite = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_quant_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_quant_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

Size of the tflite model before compression: 3.13 Mb
Size of the tflite model after compression: 0.59 Mb


The size of the quantized model is roughly 1/4 of the orignial one.

In [ ]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_quant_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(eval_model(interpreter, x_test, y_test))

Accuracy after 1000 images: 0.991000
Accuracy after 2000 images: 0.990000
Accuracy after 3000 images: 0.986667
Accuracy after 4000 images: 0.988500
Accuracy after 5000 images: 0.988400
Accuracy after 6000 images: 0.990000


## Conclusion

In this tutorial, we showed you how to create *sparse models* with the TensorFlow model optimization toolkit weight pruning API. Right now, this allows you to create models that take significant less space on disk. The resulting model can also be more efficiently implemented to avoid computation; in the future TensorFlow Lite will provide such capabilities.

More specifically, we walked you through an end-to-end example of training a simple MNIST model that used the weight pruning API. We showed you how to convert it to the Tensorflow Lite format for mobile deployment, and demonstrated how with simple file compression the model size was reduced 5x.

We encourage you to try this new capability on your Keras models, which can be particularly important for deployment in resource-constraint environments. 

